# Clustering the protein dataset

The goal is to cluster the data with several different methods, in order to find the annotations that match the clusters thee best. 

#### TODOs


## Creating the datasets

### Protein datasets

In [1]:
from subpred.dataset import create_dataset, SUBSTRATE_KEYWORDS

df_uniprot = create_dataset(
    input_file="../data/raw/uniprot/uniprot_2022_05_evidence1-2_nofragments.tsv",
    # keywords_classes = None,
    # keywords_classes_all = SUBSTRATE_KEYWORDS,
    # keywords_filter = None,
    multi_substrate="keep",
    # outliers=outliers,
    verbose=True,
    # tax_ids_filter=[3702, 9606, 83333, 559292],
    # sequence_clustering=70,
    evidence_code=2,
    invalid_amino_acids="remove_amino_acids",
    # gene_names_only = True,
    # force_update=True,
    # remove_sequence_fragments = True,
    # force_update = False,
    tcdb_substrates_file="../data/raw/tcdb/tcdb_substrates.tsv",
    swissprot_only=False,
)
print(df_uniprot.shape)
df_uniprot.head()


Found pickle, reading...
(1021957, 16)


,gene_names,protein_names,reviewed,protein_existence,sequence,organism_id,go_ids,keyword_ids,keywords,tcdb_id,interpro,tcdb_class,tcdb_substrates,keywords_substrates,keywords_transport_related,keywords_location
Uniprot,,,,,,,,,,,,,,,,
A0A0C5B5G6,MT-RNR1,Mitochondrial-derived peptide MOTS-c (Mitochon...,reviewed,Evidence at protein level,MRWQEMGYIFYPRKLR,9606,GO:0001649; GO:0003677; GO:0005615; GO:0005634...,KW-0238; KW-0496; KW-0539; KW-0892; KW-1185; K...,DNA-binding;Mitochondrion;Nucleus;Osteogenesis...,NaN,NaN,0.0,NaN,,,
A0A1B0GTW7,CIROP LMLN2,Ciliated left-right organizer metallopeptidase...,reviewed,Evidence at protein level,MLLLLLLLLLLPPLVLRVAASRCLHDETQKSVSLLRPPFSQLPSKS...,9606,GO:0004222; GO:0005737; GO:0006508; GO:0007155...,KW-0025; KW-0225; KW-0325; KW-1056; KW-0378; K...,Alternative splicing;Disease variant;Glycoprot...,NaN,IPR001577;,0.0,NaN,,,
A0JNW5,BLTP3B KIAA0701 SHIP164 UHRF1BP1L,Bridge-like lipid transfer protein family memb...,reviewed,Evidence at protein level,MAGIIKKQILKHLSRFTKNLSPDKINLSTLKGEGELKNLELDEEVL...,9606,GO:0005769; GO:0005829; GO:0034498; GO:0042803...,KW-0025; KW-0175; KW-0963; KW-0967; KW-0597; K...,Alternative splicing;Coiled coil;Cytoplasm;End...,NaN,IPR026728;IPR026854;,0.0,NaN,,,
A0JP26,POTEB3,POTE ankyrin domain family member B3,reviewed,Evidence at protein level,MVAEVCSMPAASAVKKPFDLRSKMGKWCHHRFPCCRGSGKSNMGTS...,9606,NaN,KW-0025; KW-0040; KW-0175; KW-1185; KW-0677,Alternative splicing;ANK repeat;Coiled coil;Re...,NaN,IPR002110;IPR036770;IPR039497;,0.0,NaN,,,
A0PK11,CLRN2,Clarin-2,reviewed,Evidence at protein level,MPGWFKKAWYGLASLLSFSSFILIIVALVVPHWLSGKILCQTGVDL...,9606,GO:0007605; GO:0032421; GO:0060088; GO:0060171...,KW-1003; KW-0966; KW-0209; KW-0225; KW-0325; K...,Cell membrane;Cell projection;Deafness;Disease...,9.A.46.1.2,IPR026748;,9.A,NaN,,,


In [2]:
df_swissprot = df_uniprot[df_uniprot.reviewed == "reviewed"]
df_swissprot.shape

(146597, 16)

### Annotation datasets

In [3]:
from subpred.dataset import get_go_df, get_keywords_df, get_tcdb_substrates

df_kw_uniprot = get_keywords_df(df_uniprot)
df_kw_uniprot.head()


,Uniprot,keyword
0,A0A0C5B5G6,DNA-binding
1,A0A0C5B5G6,Mitochondrion
2,A0A0C5B5G6,Nucleus
3,A0A0C5B5G6,Osteogenesis
4,A0A0C5B5G6,Reference proteome


In [4]:
df_kw_uniprot_long = df_kw_uniprot.assign(dataset="keyword").rename(
    columns={"keyword": "annotation"}
)[["Uniprot", "dataset", "annotation"]]
df_kw_uniprot_long.head()


,Uniprot,dataset,annotation
0,A0A0C5B5G6,keyword,DNA-binding
1,A0A0C5B5G6,keyword,Mitochondrion
2,A0A0C5B5G6,keyword,Nucleus
3,A0A0C5B5G6,keyword,Osteogenesis
4,A0A0C5B5G6,keyword,Reference proteome


In [5]:
from subpred.go_utils import GeneOntology

go = GeneOntology("../data/raw/ontologies/go.owl")


In [6]:
df_go_uniprot = get_go_df(df_uniprot, go)
df_go_uniprot.head()


,Uniprot,go_id,go_term
0,A0A0C5B5G6,GO:0001649,osteoblast differentiation
1,A0A0C5B5G6,GO:0003677,DNA binding
2,A0A0C5B5G6,GO:0005615,extracellular space
3,A0A0C5B5G6,GO:0005634,nucleus
4,A0A0C5B5G6,GO:0005739,mitochondrion


In [7]:
df_go_uniprot_long = df_go_uniprot.melt(
    id_vars=["Uniprot"],
    value_vars=["go_id", "go_term"],
    var_name="dataset",
    value_name="annotation",
)
df_go_uniprot_long = df_go_uniprot_long[df_go_uniprot_long.dataset == "go_term"]
df_go_uniprot_long.head()


,Uniprot,dataset,annotation
4736560,A0A0C5B5G6,go_term,osteoblast differentiation
4736561,A0A0C5B5G6,go_term,DNA binding
4736562,A0A0C5B5G6,go_term,extracellular space
4736563,A0A0C5B5G6,go_term,nucleus
4736564,A0A0C5B5G6,go_term,mitochondrion


In [8]:
# TODO TCDB classes
df_tcdb_uniprot = get_tcdb_substrates(df_uniprot)
df_tcdb_uniprot = df_tcdb_uniprot.assign(
    tcdb_class=df_tcdb_uniprot.tcdb_id.apply(lambda x: x[:3])
)
df_tcdb_uniprot.head()


,Uniprot,tcdb_id,chebi_id,chebi_term,tcdb_class
0,A6NI61,1.N.2.1.1,CHEBI:25367,molecule,1.N
1,A6NKB5,9.A.80.1.1,CHEBI:24636,proton,9.A
2,O00168,1.A.27.1.8,CHEBI:22563,anion,1.A
3,O00337,2.A.41.2.3,CHEBI:9175,sodium(1+),2.A
4,O00337,2.A.41.2.3,CHEBI:9893,uridine,2.A


In [9]:
df_tcdb_uniprot_long = df_tcdb_uniprot.melt(
    id_vars=["Uniprot"],
    value_vars=["tcdb_id", "chebi_id", "chebi_term"],
    var_name="dataset",
    value_name="annotation",
)
df_tcdb_uniprot_long.head()


,Uniprot,dataset,annotation
0,A6NI61,tcdb_id,1.N.2.1.1
1,A6NKB5,tcdb_id,9.A.80.1.1
2,O00168,tcdb_id,1.A.27.1.8
3,O00337,tcdb_id,2.A.41.2.3
4,O00337,tcdb_id,2.A.41.2.3


In [10]:
import pandas as pd

df_uniprot_interpro = (
    df_uniprot.interpro.dropna().str.rstrip(";").str.split(";").explode().str.strip()
)
assert df_uniprot_interpro[~df_uniprot_interpro.str.match("IPR[0-9]{6}")].shape[0] == 0
df_uniprot_interpro = df_uniprot_interpro.reset_index(
    drop=False
)  # [df_uniprot_interpro.isnull()]
df_uniprot_interpro


,Uniprot,interpro
0,A0A1B0GTW7,IPR001577
1,A0JNW5,IPR026728
2,A0JNW5,IPR026854
3,A0JP26,IPR002110
4,A0JP26,IPR036770
...,...,...
4190605,X5MFI4,IPR029044
4190606,X5MI49,IPR008630
4190607,X5MI49,IPR029044
4190608,X5MPI5,IPR008630


In [11]:
df_uniprot_interpro_long = df_uniprot_interpro.assign(dataset="interpro").rename(
    columns={"interpro": "annotation"}
)[["Uniprot", "dataset", "annotation"]]
df_uniprot_interpro_long.head()


,Uniprot,dataset,annotation
0,A0A1B0GTW7,interpro,IPR001577
1,A0JNW5,interpro,IPR026728
2,A0JNW5,interpro,IPR026854
3,A0JP26,interpro,IPR002110
4,A0JP26,interpro,IPR036770


In [12]:
df_uniprot_organism_long = (
    df_uniprot.organism_id.reset_index(drop=False)
    .drop_duplicates()
    .assign(dataset="organism_id")
    .rename(columns={"organism_id": "annotation"})[["Uniprot", "dataset", "annotation"]]
)


### Dataframe containing all annotations

In [13]:
df_annotations = pd.concat(
    [
        df_kw_uniprot_long,
        df_go_uniprot_long,
        df_tcdb_uniprot_long,
        df_uniprot_interpro_long,
        df_uniprot_organism_long,
    ],
    ignore_index=True,
)

df_annotations = df_annotations[~df_annotations.duplicated()]

assert df_annotations[df_annotations.Uniprot.isnull()].shape[0] == 0
assert df_annotations[df_annotations.dataset.isnull()].shape[0] == 0
assert df_annotations[df_annotations.annotation.isnull()].shape[0] == 0

df_annotations.head()


,Uniprot,dataset,annotation
0,A0A0C5B5G6,keyword,DNA-binding
1,A0A0C5B5G6,keyword,Mitochondrion
2,A0A0C5B5G6,keyword,Nucleus
3,A0A0C5B5G6,keyword,Osteogenesis
4,A0A0C5B5G6,keyword,Reference proteome


## TODOs

- Make AAC, PAAC faster
- Should I only keep proteins for which all/most types of annotation are available?
    - Most proteins do not have TCDB id, so that would not be feasible
- Add non-Uniprot GO terms?
    - Fewer terms, but we know the qualifiers and evidence codes
- Encode sequences
    - AAC, PAAC
    - Seqvec
    - Internal encoding of clustering program
- Calculate clusters
    - CD-Hit
    - MMseqs2 (used by PDB) (TODO install in env, export new env)
        - Also provides search functionality, for example for annotating organism. Can this be used to predict things with homology?
    - Sklearn unsupervised learning
- Analysis
    - Enrichment analysis of clusters, sort
        - GSEA etc.
        - Hypergeometric test
    - Rand index
- Only use Protein level evidence proteins?
- PCA
    - https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3982804/


In [14]:
df_uniprot.protein_existence.value_counts()

Evidence at transcript level    772752
Evidence at protein level       249205
Name: protein_existence, dtype: int64

## Trying PAAC + sklearn

TODO implement, test

In [2]:
from subpred.compositions import calculate_aac, calculate_paac

aac_new = calculate_aac(df_uniprot.sequence, n_threads=-1)
paac_new = calculate_paac(df_uniprot.sequence, n_threads=-1)


In [60]:
%timeit aac_new = calculate_comp(df_uniprot.sequence,k=1, n_threads = -1)

6.3 s ± 291 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Trying mmseqs2:

- Most important parameters: https://github.com/soedinglab/MMseqs2/wiki#clustering-criteria
- Output format: Long TSV TODO write parser
- Disadvantage: No pairwise scores, no PCA plots

In [20]:
# Fasta export TODO temp file
from subpred.fasta import write_fasta

fasta_file_name = "/home/ad/tmp.fasta"

df_uniprot_test = df_uniprot[
    (df_uniprot.reviewed == "reviewed")
    & (df_uniprot.protein_existence == "Evidence at protein level")
]
df_uniprot_test = df_uniprot_test.sequence.reset_index(drop=False)
df_uniprot_test.Uniprot = df_uniprot_test.Uniprot.apply(lambda x: ">" + x)
print(df_uniprot_test.shape)
df_uniprot_test = df_uniprot_test[~df_uniprot_test.duplicated()]
print(df_uniprot_test.shape)
fasta_data = list(df_uniprot_test.to_records(index=False))

write_fasta(fasta_file_name, fasta_data)


AttributeError: 'DataFrame' object has no attribute 'reviewed'

mmseqs easy-cluster tmp.fasta tmp_out.fasta tmp


--min-seq-id 0.5 -c 0.8 --cov-mode 1

## Trying Seqvec:

hdf needs pytables depencency

In [ ]:
import pandas as pd

df_secvec = pd.read_hdf("/home/ad/seqvec_test/uniref50_v2/weights.hdf5")


ValueError: Dataset(s) incompatible with Pandas data types, not table, or no datasets found in HDF5 file.

In [ ]:
import biovec


ModuleNotFoundError: No module named 'fasta'

In [ ]:
from allennlp.commands.elmo import ElmoEmbedder
from pathlib import Path

model_dir = Path("/home/ad/seqvec_test/uniref50_v2")
weights = model_dir / "weights.hdf5"
options = model_dir / "options.json"
embedder = ElmoEmbedder(options, weights, cuda_device=0)  # cuda_device=-1 for CPU

seq = "SEQWENCE"  # your amino acid sequence
embedding = embedder.embed_sentence(list(seq))  # List-of-Lists with shape [3,L,1024]


seq1 = "SEQWENCE"  # your amino acid sequence
seq2 = "PROTEIN"
seqs = [list(seq1), list(seq2)]
seqs.sort(key=len)  # sorting is crucial for speed
embedding = embedder.embed_sentences(
    seqs
)  # returns: List-of-Lists with shape [3,L,1024]


import torch

residue_embd = torch.tensor(embedding).sum(dim=0)  # Tensor with shape [L,1024]
protein_embd = (
    torch.tensor(embedding).sum(dim=0).mean(dim=0)
)  # Vector with shape [1024]


ValueError: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 40 from PyObject